In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.simplefilter('ignore')

In [5]:
ob_df = pd.read_csv('/Users/seanyu_mac/Downloads/아워박스_통합관리_조회_초안 - 시트1 (9).csv')
fs_df = pd.read_excel('/Users/seanyu_mac/Downloads/주문내역_2024-05-12~2025-05-12 (2).xlsx')
code_df = pd.read_csv('/Users/seanyu_mac/Downloads/store_t_product_option_export_2025-05-12_160047.csv')

In [7]:
# ob_df의 주문서 상태의 html 코드 없애기
ob_df['주문서상태'] = ob_df['주문서상태'].str.replace("<span class='square square-primary'>","")
ob_df['주문서상태'] = ob_df['주문서상태'].str.replace("</span>","")
ob_df['주문서상태'] = ob_df['주문서상태'].str.replace("<span class='square square-warning'>","")
ob_df['주문서상태'].unique().tolist()

['정상', '반품']

In [9]:
# ob_df의 품목명 상태의 html 코드 없애기
ob_df['품목명'] = ob_df['품목명'].apply(lambda x: x.split('</span> ')[1].split('</')[0].strip())
ob_df['품목명'].unique().tolist()

['[YDY] 포스트바이오틱스 GX (4EA)',
 '[YDY] 폴라초임계오메가3 (3EA)',
 '[YDY] 폴라초임계오메가3 (12EA)',
 '[YDY] 포스트바이오틱스 GX (2EA)',
 '[YDY] 폴라초임계오메가3 (6EA)',
 '[YDY] 퓨어 레드큐민 (4EA)',
 '[YDY] 마그듀오 K2_R1',
 '[YDY] 폴라초임계오메가3',
 '[YDY] 프로바이오 에스엘비(SLB)',
 '[YDY] 코랄칼마디 K2_R1',
 '[YDY] 콜라겐 비오틴 듀오 (2EA)',
 '[파이토알엑스] 이뮨 빌베리 (8EA)',
 '[YDY] 퓨어 레드큐민 (3EA)',
 '[YDY] 퓨어 레드큐민 (6EA)',
 '[YDY] 파이토 레드큐민 (6EA)',
 '[YDY] 파이토 레드큐민 (3EA)',
 '[파이토알엑스] 이뮨 빌베리 (12EA)',
 '[파이토알엑스] 이뮨 빌베리 (4EA)',
 '[YDY] 파이토 레드큐민 (12EA)',
 '[YDY] 퓨어 레드큐민 (12EA)',
 '[파이토알엑스] 이뮨 빌베리 (2EA)',
 '[YDY] 포스트바이오틱스 GX (12EA)',
 '[YDY] 퓨어 리포좀 비타민C',
 '[YDY] 포스트바이오틱스 GX (8EA)',
 '[YDY] 파이토 레드큐민',
 '[YDY] 포스트바이오틱스 GX']

In [11]:
# fs_df 에서 테스트 상품은 제거
fs_df = fs_df.drop(fs_df[fs_df['상품명'].str.contains('테스트')].index)
fs_df = fs_df.drop(fs_df[fs_df['상품명'].str.contains('QA')].index)
fs_df = fs_df.drop(fs_df[fs_df['옵션명'].str.contains('QA')].index)

In [13]:
temp_df = pd.merge(fs_df, code_df[['option_name', 'fulfillment_item_code']], left_on='옵션명', right_on='option_name', how='left')
temp_df.loc[temp_df['fulfillment_item_code'].isna(), '옵션명'].unique().tolist()


temp_df.loc[temp_df['옵션명'] == '마그듀오', 'fulfillment_item_code'] = 202650000001
temp_df.loc[temp_df['옵션명'] == '코랄칼마디', 'fulfillment_item_code'] = 201807000036
temp_df.loc[temp_df['옵션명'] == '썬비타민D3&K2', 'fulfillment_item_code'] = 201067000008
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/형제약사] YDY 썬비타민D3 2500IU 1박스(60일분)', 'fulfillment_item_code'] = 201067000008
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/친한] YDY 코랄칼마디 1박스(1개월분) / 칼슘 마그네슘 비타민D', 'fulfillment_item_code'] = 201807000036
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/친한] YDY 썬비타민D3 2500IU 1박스(60일분)', 'fulfillment_item_code'] = 201067000008
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/친한] YDY 마그듀오 1박스(1개월분)', 'fulfillment_item_code'] = 202650000001
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/고상온약사] YDY 썬비타민D3 2500IU 1박스(60일분)', 'fulfillment_item_code'] = 201067000008
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/위키약사] YDY 코랄칼마디 1박스(1개월분) / 칼슘 마그네슘 비타민D', 'fulfillment_item_code'] = 201807000036
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/약사임당] YDY 코랄칼마디 1박스(1개월분) / 칼슘 마그네슘 비타민D', 'fulfillment_item_code'] = 201807000036
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/고상온약사] YDY 코랄칼마디 1박스(1개월분) / 칼슘 마그네슘 비타민D', 'fulfillment_item_code'] = 201807000036
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/남기헌약사] YDY 코랄칼마디 1박스(1개월분) / 칼슘 마그네슘 비타민D', 'fulfillment_item_code'] = 201807000036
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/이재연약사] YDY 코랄칼마디 1박스(1개월분) / 칼슘 마그네슘 비타민D', 'fulfillment_item_code'] = 201807000036
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/안재돈약사] YDY 코랄칼마디 1박스(1개월분) / 칼슘 마그네슘 비타민D', 'fulfillment_item_code'] = 201807000036
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/고상온약사] YDY 마그듀오 1박스(1개월분)', 'fulfillment_item_code'] = 202650000001
temp_df.loc[temp_df['옵션명'] == '[스페셜할인코드/남기헌약사] YDY 마그듀오 1박스(1개월분)', 'fulfillment_item_code'] = 202650000001
temp_df.loc[temp_df['옵션명'] == '풀스펙멀티비타민', 'fulfillment_item_code'] = 201067000013

fs_df = temp_df

In [15]:
fs_df[fs_df['fulfillment_item_code'].isna()]

,결제 번호,상품 번호,주문 번호,결제 완료일,주문 상태,상품명,옵션명,수량,정상가,판매가,...,배송비,도서산간 배송비,취소 완료일,판매자 취소일,반품 요청일,수거 요청일,수거 완료일,반품 완료일,option_name,fulfillment_item_code


In [17]:
fs_df['fulfillment_item_code'] = fs_df['fulfillment_item_code'].astype(int)

In [19]:
fs_df['결제 완료일'] = fs_df['결제 완료일'].dt.strftime('%Y-%m-%d %H:%M:%S')
ob_df['주문일시'] = pd.to_datetime(ob_df['주문일시'])
ob_df['주문일시'] = ob_df['주문일시'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [21]:
temp_df = pd.merge(ob_df, fs_df[['결제 완료일', 'fulfillment_item_code', '옵션명', '상품명', '상품 번호']], left_on=['주문일시','업체품목코드'], right_on=['결제 완료일','fulfillment_item_code'], how='left')

In [23]:
temp_df['도서산간여부'] = temp_df['도서산간여부'].apply(lambda x: x.split("square-gray'>")[1].split('<')[0]) 

In [25]:
temp_df['채널상품명'] = temp_df['상품명']
temp_df['옵션1'] = temp_df['옵션명']
temp_df['채널아이디'] = temp_df['상품 번호']

del temp_df['결제 완료일']
del temp_df['fulfillment_item_code']
del temp_df['옵션명']
del temp_df['상품명']
del temp_df['상품 번호']

In [27]:
types = []
temp_df.loc[temp_df['채널상품명'].isna(), '채널상품명'] = '없음'

for i in temp_df['채널상품명']:
    if '공동' in i:
        types.append('공동구매')
    elif '스페셜' in i:
        types.append('친한구매')
    elif '임직' in i:
        types.append('임직원몰 상품')
    else:
        types.append('없음')

temp_df['구분'] = types

In [29]:
temp_df['주문번호'] = temp_df['주문번호'].apply(lambda x: x[:5])

In [31]:
temp_df.to_excel(f'/Users/seanyu_mac/Downloads/아워박스_업데이트_{datetime.now()}.xlsx')